In [33]:
import mne
import os
from mne.preprocessing import create_eog_epochs, EOGRegression

In [34]:
# load raw
subject_id = 1
tasktime = 1 # 1-pre, 2-post
raw_path = os.path.join('..', '..', '..', 'data', str(subject_id), 'raw_' + str(subject_id) + '_' + str(tasktime) + '.fif')
raw = mne.io.read_raw_fif(raw_path, preload=True)

Opening raw data file ..\..\..\data\1\raw_1_1.fif...
    Range : 0 ... 3865075 =      0.000 ...   805.224 secs
Ready.
Reading 0 ... 3865075  =      0.000 ...   805.224 secs...


C:\Users\Tianyi Zheng\AppData\Local\Temp\ipykernel_27696\2193600228.py:5: RuntimeWarning: This filename (..\..\..\data\1\raw_1_1.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw_fif(raw_path, preload=True)


In [35]:
# Pick EEG and EOG channels
# raw.pick_types(eeg=True, eog=True, exclude=[])

# Set EEG reference and filter data
raw.set_eeg_reference('average')
# Highpass filter to eliminate slow drifts
raw.filter(0.3, None, picks="eeg")

# Create EOG epochs
eog_epochs = create_eog_epochs(raw)  # This automatically detects EOG events
eog_epochs.apply_baseline((0, 0))  # Baseline correction, if necessary

# Perform regression to get correction coefficients
model = EOGRegression(picks="eeg", picks_artifact="eog").fit(eog_epochs)

# Apply the regression to remove EOG artifacts
raw_corrected = model.apply(raw)

# Optionally, save the cleaned data
# raw_corrected.save('path_to_save_cleaned_raw.fif', overwrite=True)

# Visualize the effect
raw.plot(title="Before EOG correction")
raw_corrected.plot(title="After EOG correction")

EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Filtering raw data in 1 contiguous segment
Setting up high-pass filter at 0.3 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal highpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.30
- Lower transition bandwidth: 0.30 Hz (-6 dB cutoff frequency: 0.15 Hz)
- Filter length: 52801 samples (11.000 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    2.2s


Using EOG channel: EOG
EOG channel index for this subject is: [32]
Filtering the data to remove DC offset to help distinguish blinks from saccades
Selecting channel EOG for blink detection
Setting up band-pass filter from 1 - 10 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 1.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 0.75 Hz)
- Upper passband edge: 10.00 Hz
- Upper transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 10.25 Hz)
- Filter length: 48000 samples (10.000 s)

Now detecting blinks and generating corresponding events
Found 79 significant peaks
Number of EOG events detected: 79
Not setting metadata
79 matching events found
No baseline correction applied
Using data from preloaded Raw for 79 events and 4801 original time points ...
0 bad epochs dropped
Applying baseline correction (m